<a href="https://colab.research.google.com/github/baohl00/OphNER/blob/main/%5BOphNER%5D_Annotate_BIO_for_1_sentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import NaN
import pandas as pd

'''
  File: /content/JSON-MIN-greaterthan512_7-697tasks-20230331.json
        /content/JSON-MIN-ALL-S-20230407.json

'''

df = pd.read_json("/content/JSON-MIN-ALL-S-20230412.json")
df = df.drop(columns=['news_#',	'title', 'sents'])
df = df.dropna()
df

,news_id,sub_content_id,sub_content,sents_1,sents_2,sent_i,text,id,label,annotator,annotation_id,created_at,updated_at,lead_time
0,176,1.0,"According to a small, randomized trial conduct...","According to a small, randomized trial conduct...",According to a small randomized trial conducte...,1,According to a small randomized trial conducte...,5078,"[{'start': 51, 'end': 85, 'text': 'the Univers...",1,4156,2023-04-07 14:42:54.542580+00:00,2023-04-07 14:42:54.542580+00:00,1944.571
1,176,1.0,"According to a small, randomized trial conduct...","According to a small, randomized trial conduct...",According to a small randomized trial conducte...,2,AMD is the main cause of vision loss in the el...,5079,"[{'start': 0, 'end': 3, 'text': 'AMD', 'labels...",1,4157,2023-04-07 14:46:01.416753+00:00,2023-04-07 14:46:01.416753+00:00,10.607
2,176,1.0,"According to a small, randomized trial conduct...","According to a small, randomized trial conduct...",According to a small randomized trial conducte...,3,AMD affects your central field of vision and c...,5080,"[{'start': 0, 'end': 3, 'text': 'AMD', 'labels...",1,4158,2023-04-07 14:46:15.605779+00:00,2023-04-07 14:46:15.605779+00:00,13.045
3,176,1.0,"According to a small, randomized trial conduct...","According to a small, randomized trial conduct...",According to a small randomized trial conducte...,4,The researchers discovered that eating 28 gram...,5081,"[{'start': 111, 'end': 122, 'text': 'for 90 da...",1,4159,2023-04-07 14:46:35.785127+00:00,2023-04-07 14:46:35.785127+00:00,18.218
5,176,1.0,"According to a small, randomized trial conduct...","According to a small, randomized trial conduct...",According to a small randomized trial conducte...,6,The pigments lutein and zeaxanthin which incre...,5083,"[{'start': 13, 'end': 19, 'text': 'lutein', 'l...",1,4161,2023-04-07 14:47:07.410126+00:00,2023-04-07 14:47:07.410126+00:00,26.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,198,7.0,Other Diseases\nSickle cell disease is one of ...,Other Diseases.\nSickle cell disease is one of...,Other Diseases.\nSickle cell disease is one of...,3,OCTA features such as blood vessel diameter an...,6149,"[{'start': 0, 'end': 4, 'text': 'OCTA', 'label...",1,5171,2023-04-10 07:09:45.165956+00:00,2023-04-10 07:09:45.166912+00:00,38.819
1713,198,7.0,Other Diseases\nSickle cell disease is one of ...,Other Diseases.\nSickle cell disease is one of...,Other Diseases.\nSickle cell disease is one of...,4,Algorithms have also been able to differentiat...,6150,"[{'start': 56, 'end': 93, 'text': 'mild sickle...",1,5172,2023-04-10 07:10:15.918122+00:00,2023-04-10 07:10:15.918122+00:00,29.227
1715,198,7.0,Other Diseases\nSickle cell disease is one of ...,Other Diseases.\nSickle cell disease is one of...,Other Diseases.\nSickle cell disease is one of...,6,Models have accurately determined a patients a...,6152,"[{'start': 101, 'end': 127, 'text': 'a single ...",1,5174,2023-04-10 07:10:25.782135+00:00,2023-04-10 07:10:25.782135+00:00,5.105
1727,198,8.0,Limitations and Conclusion\nAlthough the diagn...,Limitations and Conclusion.\nAlthough the diag...,Limitations and Conclusion.\nAlthough the diag...,11,The National Institutes of Health through its ...,6164,"[{'start': 0, 'end': 33, 'text': 'The National...",1,5186,2023-04-10 07:11:57.856827+00:00,2023-04-10 07:11:57.856956+00:00,5.970


In [ ]:
df.columns

Index(['news_id', 'sub_content_id', 'sub_content', 'sents_1', 'sents_2',
       'sent_i', 'text', 'id', 'label', 'annotator', 'annotation_id',
       'created_at', 'updated_at', 'lead_time'],
      dtype='object')

In [ ]:
df[['text', 'label']]

,text,label
0,According to a small randomized trial conducte...,"[{'start': 51, 'end': 85, 'text': 'the Univers..."
1,AMD is the main cause of vision loss in the el...,"[{'start': 0, 'end': 3, 'text': 'AMD', 'labels..."
2,AMD affects your central field of vision and c...,"[{'start': 0, 'end': 3, 'text': 'AMD', 'labels..."
3,The researchers discovered that eating 28 gram...,"[{'start': 111, 'end': 122, 'text': 'for 90 da..."
5,The pigments lutein and zeaxanthin which incre...,"[{'start': 13, 'end': 19, 'text': 'lutein', 'l..."
...,...,...
1712,OCTA features such as blood vessel diameter an...,"[{'start': 0, 'end': 4, 'text': 'OCTA', 'label..."
1713,Algorithms have also been able to differentiat...,"[{'start': 56, 'end': 93, 'text': 'mild sickle..."
1715,Models have accurately determined a patients a...,"[{'start': 101, 'end': 127, 'text': 'a single ..."
1727,The National Institutes of Health through its ...,"[{'start': 0, 'end': 33, 'text': 'The National..."


In [ ]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll))
    return results

In [ ]:
pd1_word = []
pd1_label = []
pd1_sentence_id = []

sentence = list(df['text'].values)
label = list(df['label'].values)

for i in range(len(sentence)):
    sentence_id = i
    sentence_i = sentence[i]
    label_i = label[i]
    sentence_word = sentence_i.split(" ")
    list_label = ['O'] * len(sentence_word)
    for j in label_i:
        labeled_word = j['text'].split(" ")
        index = find_sub_list(labeled_word, sentence_word)
        for k in index:
            if k[1] - k[0] == 1:
                list_label[k[0]] = 'B-' + j['labels'][0]
            else:
                for l in range(k[0],k[1]):
                    if l == k[0]:
                        list_label[l] = 'B-' + j['labels'][0]
                    else:
                        list_label[l] = 'I-' + j['labels'][0]
    pd1_word += sentence_word
    pd1_label += list_label
    pd1_sentence_id += [sentence_id+20000] * len(sentence_word)

df_1sen = pd.DataFrame({'word': pd1_word,
                        'label': pd1_label,
                        'sentence_id': pd1_sentence_id})

df_1sen

,word,label,sentence_id
0,According,O,20000
1,to,O,20000
2,a,O,20000
3,small,O,20000
4,randomized,O,20000
...,...,...,...
16720,diseases,O,20739
16721,will,O,20739
16722,begin,O,20739
16723,to,O,20739


In [ ]:
train = df_1sen[df_1sen['sentence_id']<=20675]
test = df_1sen[df_1sen['sentence_id']>20675]

print(len(train), len(test))

train.to_csv('train_2804.csv')
test.to_csv('test_2804.csv')

15209 1516
